In [1]:
from src.utils.load_data import *

ratings = load_ratings()
movies = load_movies()
tags = load_tags()
links = load_links()
genome_tags = load_genome_tags()
genome_scores = load_genome_scores()

ParserError: Error tokenizing data. C error: Calling read(nbytes) on source failed. Try engine='python'.

In [ ]:
ratings.head()

,userId,movieId,rating,timestamp
0,1,2,3.5,2005-04-02 23:53:47
1,1,29,3.5,2005-04-02 23:31:16
2,1,32,3.5,2005-04-02 23:33:39
3,1,47,3.5,2005-04-02 23:32:07
4,1,50,3.5,2005-04-02 23:29:40


In [ ]:
movies.head()

,title,genres
movieId,,
1,Toy Story (1995),"[Adventure, Animation, Children, Comedy, Fantasy]"
2,Jumanji (1995),"[Adventure, Children, Fantasy]"
3,Grumpier Old Men (1995),"[Comedy, Romance]"
4,Waiting to Exhale (1995),"[Comedy, Drama, Romance]"
5,Father of the Bride Part II (1995),[Comedy]


In [ ]:
tags.head()

,userId,movieId,tag,timestamp
0,18,4141,Mark Waters,2009-04-24 18:19:40
1,65,208,dark hero,2013-05-10 01:41:18
2,65,353,dark hero,2013-05-10 01:41:19
3,65,521,noir thriller,2013-05-10 01:39:43
4,65,592,dark hero,2013-05-10 01:41:18


In [ ]:
links.head()

,imdbId,tmdbId
movieId,,
1,114709,862.0
2,113497,8844.0
3,113228,15602.0
4,114885,31357.0
5,113041,11862.0


In [ ]:
genome_tags.head()

,tag
tagId,
1,007
2,007 (series)
3,18th century
4,1920s
5,1930s


In [ ]:
genome_scores.head()

,movieId,tagId,relevance
0,1,1,0.02500
1,1,2,0.02500
2,1,3,0.05775
3,1,4,0.09675
4,1,5,0.14675


In [1]:
from src.utils.load_data import *
from src.utils.processing import *
from sklearn.naive_bayes import GaussianNB

N = 900
N_UNIGRAMS = 3000
data = {
	"negative": load_negative(N),
	"positive": load_positive(N)
}
# print(data['positive'][0])

split_data = split_unigrams(data)
# print(split_data['positive'][0])

set_data = get_unigram_sets(split_data)
# print(set_data['positive'][0])

top_u = get_top_unigrams(N_UNIGRAMS, split_data)
print(top_u[:20])

mtx = get_unigram_presence_matrices(set_data, top_u)
# print(mtx['positive'][0])

['.', '!', 'one', 'like', '?', 'good', 'even', 'would', 'see', 'time', 'really', 'story', 'get', '-', 'much', 'people', 'great', 'first', 'also', 'well']


In [2]:
import pandas as pd

TRAIN_SIZE = N*2//3
TEST_SIZE = N//3

x_train = mtx['positive'][:TRAIN_SIZE]
x_train.extend(mtx['negative'][:TRAIN_SIZE])
y_train = [1 if i < TRAIN_SIZE else 0 for i in range(TRAIN_SIZE*2)]
# print(x_train[0])

x_test = mtx['positive'][TRAIN_SIZE : TRAIN_SIZE+TEST_SIZE]
x_test.extend(mtx['negative'][TRAIN_SIZE : TRAIN_SIZE+TEST_SIZE])
y_test = [1 if i < TEST_SIZE else 0 for i in range(TEST_SIZE*2)]

x_train = pd.DataFrame(x_train)
y_train = np.array(y_train)
# print(x_train.shape)
# print(y_train.shape)

x_test = pd.DataFrame(x_test)
y_test = np.array(y_test)

nb = GaussianNB()
nb.fit(x_train, y_train)
y_pred = nb.predict(x_test)
n_correct = (y_test == y_pred).sum()
print(f'Accuracy: {n_correct*100/x_test.shape[0]}% {n_correct}/{x_test.shape[0]}')

Accuracy: 70.83333333333333% 425/600
